In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict

sys.path.insert(0, '../')
from relative_prevalence_benchmark.gpu_utils import restrict_GPU_pytorch
restrict_GPU_pytorch('2')

import torch
from torch import nn

from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader, get_eval_loader
from wilds.common.grouper import CombinatorialGrouper

import torch
import torchvision.transforms as transforms
from torch import nn

from tqdm.notebook import tqdm

sys.path.insert(0, '../wilds_source/examples/')
from transforms import initialize_transform, getBertTokenizer
from algorithms.initializer import initialize_algorithm

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

Using GPU:2


In [4]:
full_dataset = get_dataset(dataset='civilcomments', download=False)
   
config_dict = pickle.load(open('../wilds_source/cc_config.pkl', 'rb'))
config = Namespace(**config_dict)
config.algorithm = 'ERM'

train_transform = initialize_transform(transform_name=config.transform,
                                      config=config,
                                      dataset=full_dataset,
                                      is_training=True)
eval_transform = initialize_transform(transform_name=config.transform,
                                      config=config,
                                      dataset=full_dataset,
                                      is_training=False)

In [ ]:
train_data = full_dataset.get_subset('train', transform=train_transform)
train_loader = get_eval_loader('standard', train_data, batch_size=32)

train_grouper = CombinatorialGrouper(dataset=full_dataset, groupby_fields=config.groupby_fields)

In [2]:
datasets = defaultdict(dict)
for split in full_dataset.split_dict.keys():
    if split=='train':
        transform = train_transform
        verbose = True
    elif split == 'val':
        transform = eval_transform
        verbose = True
    else:
        transform = eval_transform
        verbose = False
    # Get subset
    datasets[split]['dataset'] = full_dataset.get_subset(
        split,
        frac=config.frac,
        transform=transform)

    if split == 'train':
        datasets[split]['loader'] = get_train_loader(
            loader=config.train_loader,
            dataset=datasets[split]['dataset'],
            batch_size=config.batch_size,
            uniform_over_groups=config.uniform_over_groups,
            grouper=train_grouper,
            distinct_groups=config.distinct_groups,
            n_groups_per_batch=config.n_groups_per_batch,
        **config.loader_kwargs)
else:
    datasets[split]['loader'] = get_eval_loader(
        loader=config.eval_loader,
        dataset=datasets[split]['dataset'],
        grouper=train_grouper,
        batch_size=config.batch_size,
        **config.loader_kwargs)

# Set fields
datasets[split]['split'] = split
datasets[split]['name'] = full_dataset.split_names[split]
datasets[split]['verbose'] = verbose

NameError: name 'full_dataset' is not defined

In [3]:
alg = initialize_algorithm(config, datasets, train_grouper)
alg.load_state_dict(torch.load('../best_model.pth')['algorithm'])
alg.model.cuda()
alg.model.eval()
alg.eval()

NameError: name 'initialize_algorithm' is not defined

In [38]:
all_feats = []
all_annotations = []
it = iter(train_loader)
for batch in tqdm(it):
    jj = alg.model.distilbert(batch[0][:,:,0].cuda(), batch[0][:,:,1].cuda())['last_hidden_state']
    kk = jj[:,0].detach().cpu()
    all_feats.append(np.array(kk))
    all_annotations.append(batch[1])

  0%|          | 0/8408 [00:00<?, ?it/s]

torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300,

torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300,

torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300,

torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300,

...

torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300,

torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300,

torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300,

torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300,

torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300, 768])
torch.Size([32, 300, 768]) torch.Size([1, 300,

In [39]:
# concatenate all_feats into an array
all_feats = np.concatenate(all_feats, axis=0)
y = train_data.y_array

In [54]:
all_annotations[0]

tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 0, 1, 1, 0, 1, 1])

In [59]:
np.concatenate(all_annotations).shape

(269038, 16)

In [55]:
all_annotations = []
it = iter(train_loader)
for batch in tqdm(it):
    all_annotations.append(batch[2])

  0%|          | 0/8408 [00:00<?, ?it/s]

In [60]:
metadata = np.concatenate(all_annotations)

data_dir = '../data/real_true_labels/content_mod/'
np.save(data_dir + '/metadata', metadata)

In [40]:
annotations = np.array(['male', 'female','LGBTQ','christian','muslim', 'other_religions','black','white','identity_any','severe_toxicity','obscene', 'threat','insult','identity_attack', 'sexual_explicit', 'toxic'])
data_dir = '../data/real_true_labels/content_mod/'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

np.save(data_dir + '/vals.npz' , all_feats)
np.save(data_dir + '/metadata_col_names', annotations)
np.save(data_dir + '/metadata', metadata)
np.save(data_dir + '/observed_labels', y)

In [8]:
alg.model.distilbert

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [9]:
jj = alg.model.distilbert(batch[0][:,:,0].cuda(), batch[0][:,:,1].cuda())['last_hidden_state']
kk = jj[:,0]
kk = alg.model.pre_classifier(kk)
kk = nn.ReLU()(kk)
kk = alg.model.classifier(kk)
kk

torch.Size([10, 300, 768]) torch.Size([1, 300, 768])


tensor([[ 3.5248, -4.1342],
        [ 4.5071, -5.1383],
        [ 1.7436, -2.1969],
        [ 3.3000, -3.8558],
        [ 4.7335, -5.3430],
        [ 3.3397, -3.9665],
        [ 4.7021, -5.2412],
        [-0.8825,  1.0032],
        [ 4.8187, -5.3788],
        [ 5.1236, -5.6295]], device='cuda:0', grad_fn=<AddmmBackward>)

In [11]:
children = list(alg.model.children())
jj = children[0](batch[0][:,:,0].cuda(), batch[0][:,:,1].cuda())['last_hidden_state']
kk = nn.ReLU()(children[1](jj[:,0]))
kk = children[2](kk)

example_idx = 0
cls_idx = np.where(batch[0][example_idx][:,0] == 101)[0][0]
print(cls_idx)
kk[example_idx][cls_idx]

torch.Size([10, 300, 768]) torch.Size([1, 300, 768])


torch.Size([10, 2])

In [13]:
alg.model(batch[0].cuda())

torch.Size([10, 300, 768]) torch.Size([1, 300, 768])


tensor([[ 3.5248, -4.1342],
        [ 4.5071, -5.1383],
        [ 1.7436, -2.1969],
        [ 3.3000, -3.8558],
        [ 4.7335, -5.3430],
        [ 3.3397, -3.9665],
        [ 4.7021, -5.2412],
        [-0.8825,  1.0032],
        [ 4.8187, -5.3788],
        [ 5.1236, -5.6295]], device='cuda:0', grad_fn=<AddmmBackward>)

### Exploring examples

In [ ]:
idx = 20
full_dataset[idx], full_dataset.y_array[idx]

### Preprocess to create datasets - x, y, etc.

In [ ]:
# Filter to get two different group variables
# 11% of comments are toxic
annotations = np.array(['male', 'female','LGBTQ','christian','muslim', 'other_religions','black','white','identity_any','severe_toxicity','obscene', 'threat','insult','identity_attack', 'sexual_explicit', 'toxic'])
prevs = []
for i in range(16):
    prevs.append(np.mean(full_dataset.metadata_array.numpy()[:,i]))
    print("% ", annotations[i], ":", np.mean(full_dataset.metadata_array.numpy()[:,i]))

# prevs = np.array(prevs)
# annotations = annotations[list(np.argsort(-1*prevs))]
# prevs = prevs[list(np.argsort(-1*prevs))]
# for ann, prev in zip(annotations, prevs):
#     print("% " + ann + ":\t" + str(prev))

In [ ]:
# Get representations from train data loader. Create x 
dir(train_data)
# Create y, or s, depending on what we think

In [ ]:
jj = torch.stack([train_data[i][0] for i in range(len(train_data))])

In [ ]:
metadata = torch.stack([train_data[i][2] for i in range(len(train_data))])
y = train_data.y_array

In [ ]:
# Write out to directories

In [ ]:
features_without_attention = jj[:,:,0].numpy()
metadata = metadata.numpy()
y = y.numpy()

In [ ]:
from scipy.sparse import save_npz
import os

data_dir = '../data/real_true_labels/content_mod/'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
np.save(data_dir + '/vals.npz' , features_without_attention)
np.save(data_dir + '/metadata_col_names', annotations)
np.save(data_dir + '/metadata', metadata)
np.save(data_dir + '/observed_labels', y)

In [ ]:
full_dataset[1]